In [1]:
import numpy as np
import pandas as pd

raw_data = open('TWITTER-Real-Graph-Partial.txt','r')
preprocessed_data = open('preprocessed_data.txt','w')
for i in raw_data:
    if i.startswith('e'):
        if i[3] != ' ':
            preprocessed_data.write(i[0:4] + ',' + i[5:])
        else:
            preprocessed_data.write(i[0:3] + ',' + i[4:])
    else:
        preprocessed_data.write(i)
raw_data.close()
preprocessed_data.close()

file = pd.read_table('preprocessed_data.txt', sep=' ', header = None)

nodes_name = pd.Series(pd.unique(file[file[0] == 'n'][2]))

def BuildGraphMatrix(graph_raw):
    nodes_num = graph_raw[graph_raw[0] == 'n'].shape[0]
    edges = graph_raw[graph_raw[0] == 'e'][1]
    weights = graph_raw[graph_raw[0] == 'e'][2]
    features = graph_raw[graph_raw[0] == 'n'][2]
    label = int(graph_raw[graph_raw[0] == 'x'][1])
    
    if label == -1:
        label = np.array([0,1])
    else:
        label = np.array([1,0])

    g_matrix = np.zeros((14,14))  #need to find maximum dimension

    for index in edges.index:
        if edges[index][1] != ',':
            row = int(edges[index][0:2])
            column = int(edges[index][3:])
        else:
            row = int(edges[index][0:1])
            column = int(edges[index][2:])
        g_matrix[row-1,column-1] = weights[index]
        g_matrix[column-1,row-1] = weights[index]

    for i,word in enumerate(features):
        g_matrix[i,i] = (1 + nodes_name[nodes_name == word].index[0])/10000

    return g_matrix,label

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: read_table is deprecated, use read_csv instead.


In [2]:
graph_seg_head = 0
count = 0
g_matrix_whole = np.ndarray(shape=(len(file[file[0] == 'x'].index),14,14))
label_whole = np.zeros([len(file[file[0] == 'x'].index),2])

for n,graph_seg_end in enumerate(file[file[0] == 'x'].index):
    g_matrix,label = BuildGraphMatrix(file.loc[graph_seg_head:graph_seg_end,:])
    g_matrix_whole[n] = g_matrix
    label_whole[n] = label
    graph_seg_head = graph_seg_end + 1
    count = count + 1

    if (count%10000 == 0):
        print('Comuptation complete:',count,'/',len(file[file[0] == 'x'].index))

print('Completed!')

np.save('g_matrix.npy',g_matrix_whole)
np.save('label.npy',label_whole)

Comuptation complete: 10000 / 144033
Comuptation complete: 20000 / 144033
Comuptation complete: 30000 / 144033
Comuptation complete: 40000 / 144033
Comuptation complete: 50000 / 144033
Comuptation complete: 60000 / 144033
Comuptation complete: 70000 / 144033
Comuptation complete: 80000 / 144033
Comuptation complete: 90000 / 144033
Comuptation complete: 100000 / 144033
Comuptation complete: 110000 / 144033
Comuptation complete: 120000 / 144033
Comuptation complete: 130000 / 144033
Comuptation complete: 140000 / 144033
Completed!
